In [4]:
from SBox import *

In [5]:
# Inversible

def is_inversible(self):
    return len(set(self.sbox)) == len(self.sbox)

SBox.is_inversible = is_inversible

print(aes_sbox.is_inversible())
print([sbox.is_inversible() for sbox in des_sboxes])

True
[False, False, False, False, False, False, False, False]


In [6]:
# Avalanche effect
def hamming_distance(a, b):
    return bin(a ^ b).count('1')

def avalanche_effect(self):
    total = 0
    
    for i in range(2**self.input_len):
        for j in range(self.input_len):
            change = 1 << j
            diff = hamming_distance(self.sbox[i], self.sbox[i ^ change])
            total += (diff >= self.output_len / 2)

    return total / (self.input_len * 2**self.input_len)
SBox.avalanche_effect = avalanche_effect

print(aes_sbox.avalanche_effect())
print([(sbox.avalanche_effect()) for sbox in des_sboxes])

0.646484375
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [7]:
# Totality

def totality(self):
    all = True
    for i in range(self.input_len):
        for j in range(self.output_len):
            found = False
            for input1 in range(2**self.input_len):
                input2 = input1 ^ (1 << i)
                output1 = self.sbox[input1]
                output2 = self.sbox[input2]
                diff_output = output1 ^ output2
                diff_j = diff_output & (1 << j)
                found = found or (diff_j != 0)
            all = all and found
    return all

SBox.totality = totality

print(aes_sbox.totality())
print([sbox.totality() for sbox in des_sboxes])

True
[True, True, True, True, True, True, True, True]


In [8]:
# Strong avalanche effect

def strong_avalanche_effect(self):
    all = True
    for i in range(self.input_len):
        for j in range(self.output_len):
            count = 0
            for x in range(2**self.input_len):
                y = x ^ (1 << i)
                diff = self.sbox[x] ^ self.sbox[y]
                diff_j = diff & (1 << j)
                count += (diff_j != 0)
            all = all and (count == (2**self.input_len / 2))
    return all
SBox.strong_avalanche_effect = strong_avalanche_effect

print(aes_sbox.strong_avalanche_effect())
print([sbox.strong_avalanche_effect() for sbox in des_sboxes])

False
[False, False, False, False, False, False, False, False]


In [9]:
# Cycle decomposition

def cycle_decomposition(self):
    cycles = []
    visited = [False] * (2**self.input_len)
    for i in range(2**self.input_len):
        if not visited[i]:
            cycle = []
            j = i
            while not visited[j]:
                cycle.append(j)
                visited[j] = True
                j = self.sbox[j]
            if len(cycle) > 1:
                cycles.append(cycle)
    return cycles
SBox.cycle_decomposition = cycle_decomposition

aes_cycles = aes_sbox.cycle_decomposition()
for cycle in aes_cycles:
    print(cycle)
    print(len(cycle))
print([sbox.cycle_decomposition() for sbox in des_sboxes])

[0, 99, 251, 15, 118, 56, 7, 197, 166, 36, 54, 5, 107, 127, 210, 181, 213, 3, 123, 33, 253, 84, 32, 183, 169, 211, 102, 51, 195, 46, 49, 199, 198, 180, 141, 93, 76, 41, 165, 6, 111, 168, 194, 37, 63, 117, 157, 94, 88, 106, 2, 119, 245, 230, 142, 25, 212, 72, 82]
59
[1, 124, 16, 202, 116, 146, 79, 132, 95, 207, 138, 126, 243, 13, 215, 14, 171, 98, 170, 172, 145, 129, 12, 254, 187, 234, 135, 23, 240, 140, 100, 67, 26, 162, 58, 128, 205, 189, 122, 218, 87, 91, 57, 18, 201, 221, 193, 120, 188, 101, 77, 227, 17, 130, 19, 125, 255, 22, 71, 160, 224, 225, 248, 65, 131, 236, 206, 139, 61, 39, 204, 75, 179, 109, 60, 235, 233, 30, 114, 64, 9]
81
[4, 242, 137, 167, 92, 74, 214, 246, 66, 44, 113, 163, 10, 103, 133, 151, 136, 196, 28, 156, 222, 29, 164, 73, 59, 226, 152, 70, 90, 190, 174, 228, 105, 249, 153, 238, 40, 52, 24, 173, 149, 42, 229, 217, 53, 150, 144, 96, 208, 112, 81, 209, 62, 178, 55, 154, 184, 108, 80, 83, 237, 85, 252, 176, 231, 148, 34, 147, 220, 134, 68, 27, 175, 121, 182, 78, 47, 

In [26]:
# Branch number

block_size = 1

def block_weight(x, n):
    binary = bin(x)[2:].ljust(n,'0')
    blocks = [binary[i:i+block_size] for i in range(0, len(binary), block_size)]
    return len(binary) // block_size - blocks.count("0" * block_size)

print(block_weight(0b1010111100000000))
print(block_weight(0b1))

def branch_number(self):
    min_weight = self.input_len * 3
    for i in range(2**self.input_len):
        for j in range(2**self.input_len):
            if i != j:
                weight = block_weight(i ^ j)
                s_weight = block_weight(self.sbox[i] ^ self.sbox[j])
                if weight + s_weight < min_weight:
                    min_weight = weight + s_weight
    return min_weight
SBox.branch_number = branch_number

print(aes_sbox.branch_number())

print([sbox.branch_number() for sbox in des_sboxes])

6
1
2
[2, 2, 2, 2, 2, 2, 2, 2]
